# Fracking and Oklahoma Quakes analysis

In this analysis, I will be looking at the dataset [Oklahoma Earthquakes and Saltwater Injection Wells](https://www.kaggle.com/ksuchris2000/oklahoma-earthquakes-and-saltwater-injection-wells) to see if I can find any correlation between the earthquakes and the use of [Injection Wells](https://en.wikipedia.org/wiki/Injection_well)

### Preface

In [1]:
from datetime import datetime
import pandas as pd
import altair as alt
alt.data_transformers.enable('data_server')

DataTransformerRegistry.enable('data_server')

http://www.ogs.ou.edu/pubsscanned/openfile/OF1_2014_Murray.pdf

### Preprocessing

Preprocessing the data consists of removing unneeded rows and columns relating to the location of the well/quake as well as adding a python Datetime object as an attribute for the date of the event.

In [2]:
WELLS_PATH_RAW = 'data_fracking/raw/InjectionWells.csv'
QUAKES_PATH_RAW = 'data_fracking/raw/okQuakes.csv'
WELLS_PATH = 'data_fracking/processed/Wells.csv'
QUAKES_PATH = 'data_fracking/processed/Quakes.csv'

In [11]:
def preprocess_wells():
    wells_data = pd.read_csv(WELLS_PATH_RAW)

    wells_data = wells_data.drop(columns=[
        'API#', 'Operator ID', 'WellName', 'QQQQ', 
        'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
        'Sec', 'Twp', 'Rng'])

    years = []
    months = []
    days = []
    time = []
    well_count = []

    # add date time values
    for index, row in wells_data.iterrows():
        # If you can parse the date
        try:
            date = row['Approval Date']
            month = int(date.split('/')[0])
            day = int(date.split('/')[1])
            year = int(date.split('/')[2])
            years.append(year)
            months.append(month)
            days.append(day)
            time.append(datetime(year, month, day))

        # Else, drop the row
        except: 
            wells_data = wells_data.drop(index)

    wells_data['year'] = years
    wells_data['month'] = months
    wells_data['day'] = days
    wells_data['time'] = time

    wells_data = wells_data.sort_values('time')

    _2D_count = 0
    _2d_count = 0
    _2R_count = 0
    _2RSI_count = 0
    CDW_count = 0
    GS_count = 0

    # add well count 
    for index, row in wells_data.iterrows():
        if row['WellType'] == '2D':
            _2D_count += 1
            well_count.append(_2D_count)
        if row['WellType'] == '2d':
            _2d_count += 1
            well_count.append(_2d_count)
        elif row['WellType'] == '2R':
            _2R_count += 1
            well_count.append(_2R_count)
        elif row['WellType'] == '2RSI':
            _2RSI_count += 1
            well_count.append(_2RSI_count)
        elif row['WellType'] == 'CDW':
            CDW_count += 1
            well_count.append(CDW_count)
        elif row['WellType'] == 'GS':
            GS_count += 1
            well_count.append(GS_count)

    wells_data['well_count'] = well_count

    return wells_data.sort_values('time').reset_index(drop=True)

def preprocess_quakes():
    quakes_data = pd.read_csv(QUAKES_PATH_RAW)

    years = []
    months = []
    days = []
    time = []

    for index, row in quakes_data.iterrows():
        # If you can parse the date
        try:
            _time = row['time']
            date = _time.split('T')[0]
            year = date.split('-')[0]
            month = date.split('-')[1]
            day = date.split('-')[2]
            years.append(year)
            months.append(month)
            days.append(day)
            time.append(datetime.fromisoformat(_time.split('T')[0]))
        # Else, drop the row
        except:
            quakes_data = quakes_data.drop(index)

    quakes_data['year'] = years
    quakes_data['months'] = months
    quakes_data['days'] = days
    quakes_data['time'] = time

    return quakes_data.sort_values('time').reset_index(drop=True)

In [6]:
quakes = preprocess_quakes()
wells = preprocess_wells()

First, I want to take a look at the total number of wells over time and the amount of earthquakes over time.

In [27]:
wells_plot = alt.Chart(wells.reset_index()).mark_area().encode(
    x='time:T',
    y='index',
).properties(width=800)

quakes_plot = alt.Chart(quakes.reset_index()).mark_bar().encode(
    x='time:T',
    y='count()'
).properties(width=800, height=200)

wells_plot & quakes_plot

alt.VConcatChart(...)

Now let's take a look at where the wells are, and where the quakes are on a map.

In [41]:
states_url = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/united-states/us-albers.json'

states = alt.topo_feature(url=states_url, feature='us')

#alt.Chart(states).mark_geoshape().encode()
alt.Chart(quakes).mark_point().encode(latitude='latitude',longitude='longitude', color=alt.Value('red'))

TypeError: sequence item 1: expected str instance, int found

alt.Chart(...)

In [39]:
quakes

time  latitude   longitude  depth   mag magType   nst    gap  \
0     1973-03-17   36.0870 -106.168000   6.00  4.50      mb   NaN    NaN   
1     1973-05-25   33.9170  -90.775000   6.00   NaN     NaN   NaN    NaN   
2     1973-09-19   37.1600 -104.594000   5.00   NaN     NaN   NaN    NaN   
3     1973-09-23   37.1480 -104.571000   5.00  4.20      mb   NaN    NaN   
4     1974-02-15   36.5000 -100.693000  24.00  4.50      mb   NaN    NaN   
...          ...       ...         ...    ...   ...     ...   ...    ...   
13949 2016-09-20   37.2730  -98.069167   5.09  2.19      ml  13.0  238.0   
13950 2016-09-20   36.4372  -96.900800   6.05  2.20      ml   NaN   71.0   
13951 2016-09-20   36.4360  -96.906200   4.50  2.40      ml   NaN   66.0   
13952 2016-09-20   37.2750  -98.071833   5.29  2.28      ml  15.0  235.0   
13953 2016-09-20   36.9405  -97.906300   7.20  2.50   mb_lg   NaN   39.0   

          dmin   rms  ... horizontalError depthError magError magNst  \
0          NaN   NaN  ...             NaN        NaN      NaN    NaN   
1          NaN   NaN  ...             NaN        NaN      NaN    NaN   
2          NaN   NaN  ...             NaN        NaN      NaN    NaN   
3          NaN   NaN  ...             NaN        NaN      NaN    NaN   
4          NaN   NaN  ...             NaN        NaN      NaN    NaN   
...        ...   ...  ...             ...        ...      ...    ...   
13949  0.04743  0.04  ...            0.36       0.49    0.116   18.0   
13950  0.00400  0.32  ...            0.70       1.70    0.062   34.0   
13951  0.00100  0.20  ...            0.80       1.30    0.055   43.0   
13952  0.04962  0.05  ...            0.33       0.41    0.188   18.0   
13953  0.04800  0.35  ...            1.00       3.40    0.170    9.0   

         status  locationSource  magSource  year  months days  
0      reviewed              us         us  1973      03   17  
1      reviewed               s         us  1973      05   25  
2      reviewed              us         us  1973      09   19  
3      reviewed              us         us  1973      09   23  
4      reviewed              us         us  1974      02   15  
...         ...             ...        ...   ...     ...  ...  
13949  reviewed            ismp       ismp  2016      09   20  
13950  reviewed              us         us  2016      09   20  
13951  reviewed              us         us  2016      09   20  
13952  reviewed            ismp       ismp  2016      09   20  
13953  reviewed              us         us  2016      09   20  

[13954 rows x 25 columns]

In [25]:
alt.Chart(wells.reset_index()).mark_bar().encode(
    x='WellType:O',
    y='count()',
    color='WellType:O'
).properties(width=800, height=500)

alt.Chart(...)

In [9]:
well_type_plot = alt.Chart(wells).mark_line().encode(
    x='time:T',
    y='well_count:Q',
    color='WellType:N'
)

(well_type_plot + quakes_plot).properties(width=1000, height=500).interactive()

alt.LayerChart(...)

In [10]:
alt.Chart(wells).mark_bar().encode(
    x='time:T',
    y='count()',
).properties(width=800, height=500).interactive()

alt.Chart(...)